In [ ]:
from keras.layers import Dense, Input
from keras.models import Model

# Size of our encoded representations
encoding_dim = 32 # 32 floats: compression of factors 24.5, assuming the input is 784 floats.

# This is our input placeholder
input_img = Input(shape = (784,))

# 'Encoded' is the encoded representation of the input
encoded = Dense(encoding_dim, activation = 'relu')(input_img)

# 'Decoded' is the lossy reconstruction of the input
decoded = Dense(784, activation = 'sigmoid')(encoded)

# This model maps an input to its reconstruction
autoencoder = Model(input_img, decoded)

In [ ]:
# This model maps an input to its encoded representation
encoder = Model(input_img, encoded)

In [ ]:
# Create a placeholder for an encoded (32-dimensional) input
encoded_input = Input(shape = (encoding_dim,))
# Retrieve the last layer of the autoencoder model
decoder_layer = autoencoder.layers[-1]
# Create the decoder model
decoder = Model(encoded_input, decoder_layer(encoded_input))

In [ ]:
autoencoder.compile(optimizer = 'adadelta', loss = 'binary_crossentropy')

In [ ]:
from keras.datasets import mnist
import numpy as np
(x_train, _), (x_test, _) = mnist.load_data()

In [ ]:
x_train = x_train.astype('float32')/255
x_test = x_test.astype('float32')/255
x_train = x_train.reshape((len(x_train),np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test),np.prod(x_test.shape[1:])))
print(x_train.shape)
print(x_test.shape)

In [ ]:
autoencoder.fit(x_train, x_train, epochs = 1000, batch_size = 256, shuffle = True, validation_data = (x_test, x_test))

In [ ]:
# Encode and decode some digits
# Note that we take them from the 'test' set
encoded_imgs = encoder.predict(x_test)
decoded_imgs = decoder.predict(encoded_imgs)

In [ ]:
# Use matplotlib
import matplotlib.pyplot as plt

n = 10 # Total number of didgits to be displayed
plt.figure(figsize =(20, 4))
for i in range(n):
    # display original
    ax = plt.subplot(2, n, i+1)
    plt.imshow(x_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    
    #display reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()